Load Libraries and Function

In [57]:
import pandas as pd
import numpy as np 
import pymysql as mysql 
import warnings
import matplotlib.pyplot as plt 


Import Datasets

In [62]:
learn = pd.read_csv("learning_loss_covid_database.csv")
demos = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/demographics.csv")
epi = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv")
health = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/health.csv")
vacc = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/vaccinations.csv")
gdp = pd.read_csv("GDPForecast.csv")


IncompleteRead: IncompleteRead(1079852 bytes read, 464755 more expected)

Examine DataFrames Info: Shape, Datatypes, and Missing Values

In [ ]:
Write Data to CSV Files

In [59]:
learn.info()
demos.info() 
epi.info()
health.info() 
vacc.info()
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      41 non-null     object 
 1   loss         41 non-null     float64
 2   weeks        41 non-null     int64  
 3   deaths       41 non-null     float64
 4   gdp          41 non-null     float64
 5   private      41 non-null     float64
 6   internet     41 non-null     int64  
 7   hlo          41 non-null     float64
 8   vac          41 non-null     float64
 9   stringency   41 non-null     float64
 10  union        41 non-null     float64
 11  polity       41 non-null     int64  
 12  loggdp       41 non-null     float64
 13  logweeks     39 non-null     float64
 14  weeks2       41 non-null     int64  
 15  pop          41 non-null     int64  
 16  schooling    41 non-null     float64
 17  wbcode       41 non-null     object 
 18  region_code  41 non-null     object 
 19  hlo25     

View Dataframes and Make Note of Redundant Variables within Datasets and Between Datasets

In [ ]:
# Redundant information: deaths, gdp, vac, loggdp, logweeks, weeks2, pop, hlo25, covid25
learn.head()

In [ ]:
# Redundant information: Shows total country data, and then breaks down some countries by region or province. Need to reduce to whole country data only.
demos.head()

In [ ]:
# Needs to be reduced down to largest cummulative total for each country
epi.head()

In [ ]:
# Need to reduce down to cummulative country totals only. Some have additional region information.
health.head()

In [ ]:
# Times series data needs to be reduced to maximum cummulative numbers.
vacc.head()

In [60]:
# Needs to be trimmed to years of pandemic and forecast only
gdp.head()

,Real GDP growth (Annual percent change),1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028
0,Afghanistan,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,3.9,-2.4,-20.7,no data,no data,no data,no data,no data,no data,no data
1,Albania,2.7,5.7,2.9,1.1,2,-1.5,5.6,-0.8,-1.4,...,2.1,-3.3,8.9,4.8,3.6,3.3,3.4,3.5,3.5,3.5
2,Algeria,-5.4,3,6.4,5.4,5.6,5.6,-0.2,-0.7,-1.9,...,1,-5.1,3.4,3.2,3.8,3.1,2.5,1.9,1.7,1.7
3,Andorra,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,2,-11.2,8.3,8.8,2.1,1.5,1.5,1.5,1.5,1.5
4,Angola,2.4,-4.4,0,4.2,6,3.5,2.9,4.1,6.1,...,-0.7,-5.6,1.2,3,1.3,3.3,3.4,3.5,3.5,3.6


Updating Dataframes to Reflect only Necessary Data

Learning data needs some columns dropped.

In [ ]:
learn = learn.drop(columns = ['deaths', 'gdp', 'vac', 'loggdp', 'logweeks', 'weeks2', 'pop', 'hlo25', 'covid25'])
learn.head()

Demographics data needs to be reduced to full country stats only. Delete regional data.

In [ ]:
demos = demos[demos['location_key'].str.len() == 2]

In [ ]:
demos.head()

In [ ]:
demos['location_key'].value_counts()

COVID Epidemiology data needs to be reduced to final pandemic numbers, i.e., the maximum cummulative totals.

In [ ]:
# Drop "new" columns and retain only cummulative columns
epi = epi.drop(columns = ['new_confirmed', 'new_deceased', 'new_recovered', 'new_tested'])


In [ ]:
# Drop regional country information and retain only primary country codes
epi = epi[epi['location_key'].str.len() == 2]

In [ ]:
# Find max value, or most recent date for each country
epi = epi.groupby(by = 'location_key').max()
epi.head()

In [ ]:
epi.info()

Health data needs to be reduced to primary country ID code only. Remove regional country information.

In [ ]:
health = health[health['location_key'].str.len() == 2]
health.head()

In [ ]:
health.info()

Vaccine information needs to be reduced to cummulative numbers for each primary country code.

In [ ]:
vacc = vacc[vacc['location_key'].str.len() == 2]
vacc.head()

In [ ]:
vacc = vacc.groupby(by = 'location_key').max()
vacc.head()

GDP Information Needs to be reduced to Pandemic Years and Future Forecasts Only: 2019 onward

In [ ]:
gdp = gdp.drop([0:40])
gdp.head()

Assessing Country IDs:
Examine the number of countries available in each dataset and the format of the country IDs. Some are abbreviated and some are spelled out. 

In [ ]:
# String names for countries
# 41 countries
learn['country'].value_counts()


In [ ]:
# Abbreviations for countries
demos['location_key'].value_counts()


In [ ]:
# Abbreviations for countries
epi['location_key'].value_counts()


In [ ]:
# Abbreviations for countries
health['location_key'].value_counts()


In [ ]:
# Abbreviations for countries
vacc['location_key'].value_counts()


In [ ]:
# Country names as strings
gdp['Real GDP growth (Annual percent change)'].value_counts()

GDP and learning loss need to be updated to abbreviations.

A Primary Table needs to be made with all country names corresponding to country abbreviation IDs.
